In [9]:
%config InlineBackend.figure_format='retina' ## На маке графики выглядят четче

In [10]:
from copy import deepcopy

# Для работы с табличными данными
import pandas as pd
import numpy as np

# Подрубаем рисовалки
import matplotlib.pyplot as plt
import seaborn as sns

# Необходимые метрики и препроцессинг
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score,\
                            recall_score, roc_auc_score, roc_curve, cohen_kappa_score, fbeta_score

# Модель градиентного бустинга
from xgboost import XGBClassifier

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GroupKFold, cross_val_score

# Подрубаем progress bar
from tqdm.notebook import tqdm

# Отрубаем warning
import warnings
warnings.filterwarnings('ignore')

In [11]:
# Для интерактивных графиков (в самом конце)
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Загрузим данные

In [12]:
samples = pd.read_excel(u'20.01.23 список образцов Моча.xlsx', header=2)
samples.dropna(subset=['Gender'], inplace=True)

samples.fillna(0, inplace=True)
samples.reset_index(drop=True, inplace=True)

In [13]:
spectra1 = pd.read_csv('urine_chm001-250.csv')
spectra2 = pd.read_csv('urine_chm251-500.csv')
spectra_init = pd.concat([spectra1, spectra2], axis=0)
del(spectra1, spectra2)


spectra = spectra_init.filter(regex='^Ch')
## Чтобы использовать Initial Spectra, ничего не раскоментировать


## Чтобы использовать Sum of LED Spectra, раскоментить блок ниже
#######################################################################
# spectra = pd.DataFrame(spectra.values[:,::2]+spectra.values[:,1::2],
#                        columns=['Ch_'+str(i) for i in range(18)])
#######################################################################


## Чтобы использовать Harmonic Average Spectra, раскоментить блок ниже
#######################################################################
def TrigAVG(spec1, spec2, beta=1):
    return (1+beta**2)*spec1*spec2/(beta**2*spec1 + spec2)

spec1 = spectra.iloc[:,::2].values
spec2 = spectra.iloc[:,1::2].values
spectra = pd.DataFrame(TrigAVG(spec1, spec2, beta=0.2),
                       columns=['Ch_{}'.format(i) for i in range(18)])
#######################################################################


frame_lol = samples[['Gender','Age','Dataset']+list(samples.filter(regex=r'_a$').columns)]
spectra['Dataset'] = spectra_init.Dataset.values
spectra = pd.merge(left=spectra, right=frame_lol, how='left', on='Dataset')
# spectra.dropna(subset=samples.filter(regex=r'_a$').columns, inplace=True)
spectra.dropna(how='any', inplace=True)
# spectra = spectra.reindex(index=range(spectra.shape[0]))
spectra = pd.DataFrame(spectra.values, columns=spectra.columns)

In [14]:
Anom_Names = spectra.filter(regex='_a$').columns[:-1]

##  Добавим к исходным спектральным признакам Пол и Возраст

In [15]:
from sklearn.preprocessing import OneHotEncoder
Gender_emb = pd.DataFrame(OneHotEncoder().fit_transform(spectra.Gender.values.reshape(-1,1)).toarray(), 
                          columns=['Female', 'Male'])
Age_emb = pd.DataFrame(spectra.loc[:,'Age'])

In [16]:
## Возвращает сразу все стандартные метрики

def return_metrics(y_true, y_pred, y_prob):
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f_score = fbeta_score(y_true, y_pred, beta=0.7)
    try:
        roc_auc = roc_auc_score(y_true, y_prob)
    except ValueError: 
        roc_auc = -1
    
    return(prec, rec, f_score, roc_auc)

# Настройка вероятностей
**Для начала подберем набор гиперпараметров 
(для начала только количество и глубину деревьев) 
максимизируя ROC-AUC.**

**Цель : хотим получить алгоритмом наиболее "различимые"
оценки вероятности принадлежности обьекта классу "Болен"**

**!WARNING!** На моем скромном ноуте считалось это довольно долго (порядка 1-2 часов)

In [24]:
from itertools import product
from copy import deepcopy

xgb_params = {'n_estimators': 20, 'max_depth':  2, 'n_jobs': 6}
Depth_arr = np.arange(2,8)
Estimators_arr = np.arange(20,200,20)


Anom_Best_Dict = dict()
X = pd.concat([spectra.filter(regex='Ch_'), Gender_emb, Age_emb], axis=1).values

for anom in Anom_Names:
    
    if anom=='Bilirubin_a' or anom=='Spermatozoon_a':
        continue
    
    print("Anomaly: {}".format(anom))
    y = spectra[anom].astype(int).values
    groups = spectra.Dataset.values

    best_score=0.0
    for N_EST, MAX_DEPTH in tqdm(list(product(Estimators_arr, Depth_arr))):
        xgb_params['n_estimators'] = N_EST
        xgb_params['max_depth'] = MAX_DEPTH
        
        clf = XGBClassifier(**xgb_params)
        cur_score = cross_val_score(clf, X, y, groups=groups, cv=4, scoring='roc_auc')
        mean_score, std_score = cur_score.mean(), cur_score.std()
        
        if mean_score > best_score:
            best_score = deepcopy(mean_score)
            best_params = deepcopy(xgb_params)
            print(mean_score.round(4), std_score.round(4), best_params)
    
    Anom_Best_Dict[anom] = (best_score, best_params)
        
        
    

Anomaly: Density_a


0.7644 0.076 {'n_estimators': 20, 'max_depth': 2, 'n_jobs': 6}

Anomaly: pH_a


0.5468 0.1929 {'n_estimators': 20, 'max_depth': 2, 'n_jobs': 6}
0.6148 0.0935 {'n_estimators': 20, 'max_depth': 3, 'n_jobs': 6}
0.6498 0.0735 {'n_estimators': 20, 'max_depth': 4, 'n_jobs': 6}
0.6783 0.0649 {'n_estimators': 20, 'max_depth': 5, 'n_jobs': 6}
0.68 0.1446 {'n_estimators': 40, 'max_depth': 2, 'n_jobs': 6}
0.7244 0.069 {'n_estimators': 40, 'max_depth': 4, 'n_jobs': 6}
0.7563 0.0851 {'n_estimators': 60, 'max_depth': 4, 'n_jobs': 6}
0.7592 0.0917 {'n_estimators': 80, 'max_depth': 4, 'n_jobs': 6}

Anomaly: Protein_a


0.8139 0.0662 {'n_estimators': 20, 'max_depth': 2, 'n_jobs': 6}
0.818 0.0661 {'n_estimators': 20, 'max_depth': 3, 'n_jobs': 6}
0.8277 0.0651 {'n_estimators': 40, 'max_depth': 2, 'n_jobs': 6}
0.8384 0.0572 {'n_estimators': 60, 'max_depth': 2, 'n_jobs': 6}

Anomaly: Glucose_a


0.5637 0.0402 {'n_estimators': 20, 'max_depth': 2, 'n_jobs': 6}
0.5783 0.1108 {'n_estimators': 20, 'max_depth': 4, 'n_jobs': 6}
0.6255 0.0512 {'n_estimators': 20, 'max_depth': 6, 'n_jobs': 6}

Anomaly: Ketones_a


...

0.7007 0.0587 {'n_estimators': 20, 'max_depth': 2, 'n_jobs': 6}
0.7064 0.0759 {'n_estimators': 20, 'max_depth': 4, 'n_jobs': 6}
0.708 0.068 {'n_estimators': 20, 'max_depth': 7, 'n_jobs': 6}
0.7084 0.0503 {'n_estimators': 40, 'max_depth': 2, 'n_jobs': 6}
0.7204 0.0578 {'n_estimators': 60, 'max_depth': 2, 'n_jobs': 6}
0.7228 0.0452 {'n_estimators': 80, 'max_depth': 2, 'n_jobs': 6}

Anomaly: Ferment_a


0.6675 0.1156 {'n_estimators': 20, 'max_depth': 2, 'n_jobs': 6}
0.6977 0.1073 {'n_estimators': 20, 'max_depth': 3, 'n_jobs': 6}
0.71 0.0672 {'n_estimators': 20, 'max_depth': 5, 'n_jobs': 6}
0.7334 0.0742 {'n_estimators': 40, 'max_depth': 2, 'n_jobs': 6}
0.7456 0.0815 {'n_estimators': 60, 'max_depth': 2, 'n_jobs': 6}
0.749 0.0775 {'n_estimators': 80, 'max_depth': 2, 'n_jobs': 6}

Anomaly: Small cells_a


0.8318 0.0172 {'n_estimators': 20, 'max_depth': 2, 'n_jobs': 6}
0.8427 0.0473 {'n_estimators': 20, 'max_depth': 3, 'n_jobs': 6}

Anomaly: Pathological cylinders_a


0.7777 0.0547 {'n_estimators': 20, 'max_depth': 2, 'n_jobs': 6}
0.7866 0.0232 {'n_estimators': 20, 'max_depth': 3, 'n_jobs': 6}
0.813 0.0298 {'n_estimators': 20, 'max_depth': 5, 'n_jobs': 6}

Anomaly: Slime_a


0.8947 0.0459 {'n_estimators': 20, 'max_depth': 2, 'n_jobs': 6}
0.9063 0.0514 {'n_estimators': 20, 'max_depth': 3, 'n_jobs': 6}
0.908 0.0374 {'n_estimators': 20, 'max_depth': 4, 'n_jobs': 6}
0.9089 0.0427 {'n_estimators': 40, 'max_depth': 5, 'n_jobs': 6}
0.9127 0.0396 {'n_estimators': 40, 'max_depth': 6, 'n_jobs': 6}



In [33]:
from imblearn.metrics import sensitivity_score, specificity_score

def my_fbeta_score(y_true, y_pred, beta=1):
    sens = sensitivity_score(y_true, y_pred)
    spec = specificity_score(y_true, y_pred)
    
    return (1+beta**2)*(spec*sens)/(beta**2*spec + sens)

def return_med_metrics(y_true, y_pred, y_prob):
    sens = sensitivity_score(y_true, y_pred)
    spec = specificity_score(y_true, y_pred)
    beta = 1.2
    try:
        my_fscore = (1 + beta**2) * (spec * sens) / (beta**2 * spec + sens)
    except:
        my_fscore = -1
    try:
        roc_auc = roc_auc_score(y_true, y_prob)
    except ValueError: 
        roc_auc = -1
    
    return [sens, spec, my_fscore, roc_auc]

$$ \text{My "Medicine" F-Score:  } F_{\beta}=\left(1+\beta^{2}\right) \cdot \frac{\text { specificity } \cdot \text { sensitivity }}{\left(\beta^{2} \cdot \text { specificity }\right)+\text { sensitivity }}$$

## Настройка порога предсказания
После получения наилучших по **ROC-AUC** оценок вероятностей подберем порог предсказания максимизируя **Medicine F Beta Score**  --

 -- тригонометрическое взвешенное среднее между **Чувствительностью** и **Специфичностью** (поставив вес $\beta$ выше 1 предпочтение будет отдаваться Чувствительности)**

In [34]:
def predict(proba, thrs):
    return [1 if p>=thrs else 0 for p in proba]

In [35]:
def log_to_print(Anom_Name, Best):
    print("For anomaly {}:\n".format(Anom_Name))
    print("Чувствительность: {}\nСпецифичность: {}\n".format(round(Best.Sensitivity,3), round(Best.Specificity,3)))
    Anom = int(Best['Anom/Norm'].split('/')[0])/10
    Norm = int(Best['Anom/Norm'].split('/')[1])/10
    print("Всего больных (в среднем на тесте): {}\nВсего здоровых (в среднем на тесте): {}\n".\
          format(round(Anom,1), round(Norm,1)))
    print("Выявлено истинно больных : {}".format(round(Anom*Best.Sensitivity, 1)))
    print("Выявлено истинно здоровых : {}\n".format(round(Norm*Best.Specificity, 1)))
    print("================================================\n")
    
def log_to_file(Anom_Name, Best, filename='XGBoost_results.txt'):
    
    with open(filename, 'a') as f_res:
        f_res.write("\nFor anomaly {}:\n".format(Anom_Name))
        f_res.write("Чувствительность: {}\nСпецифичность: {}\n".format(round(Best.Sensitivity,3), round(Best.Specificity,3)))
        Anom = int(Best['Anom/Norm'].split('/')[0])/10
        Norm = int(Best['Anom/Norm'].split('/')[1])/10
        f_res.write("Всего больных (в среднем на тесте): {}\nВсего здоровых (в среднем на тесте): {}\n".format(round(Anom,1), round(Norm,1)))
        f_res.write("Выявлено истинно больных : {}\n".format(round(Anom*Best.Sensitivity, 1)))
        f_res.write("Выявлено истинно здоровых : {}\n".format(round(Norm*Best.Specificity, 1)))
        f_res.write("\n================================================\n")

    

In [36]:
## Возьмем текущую дату для добавления в название файла

from time import localtime
LT = localtime()
Year = str(LT.tm_year)[-2:]
Month = str(LT.tm_mon) if len(str(LT.tm_mon))==2 else '0'+str(LT.tm_mon)
Day = str(LT.tm_mday) if len(str(LT.tm_mday))==2 else '0'+str(LT.tm_mday)

add_to_filename = '_'.join([Year, Month, Day])
print("Local date: {}".format(add_to_filename))

Local date: 20_05_26


In [37]:
for Anom in Anom_Best_Dict.keys():

    X = pd.concat([spectra.filter(regex='Ch_'), Gender_emb, Age_emb], axis=1).values
    y = spectra[Anom].astype(int).values
    groups = spectra.Dataset.values

    CV_True_y, CV_Probs = [], []
    gkf = GroupKFold()

    for train_inds, test_inds in gkf.split(X,y,groups):
        Train_X, Test_X = X[train_inds], X[test_inds]
        Train_y, Test_y = y[train_inds], y[test_inds]

        CLF = XGBClassifier(**Anom_Best_Dict[Anom][1]).fit(Train_X, Train_y)

        Probs = CLF.predict_proba(Test_X)[:,1]
        CV_Probs.append(Probs)
        CV_True_y.append(Test_y)
        
    THRS_ARR = np.arange(0.002,1.0,0.002)

    DF = pd.DataFrame(columns=["Sensitivity", "Specificity", 
                               "Medic F-Score", "ROC-AUC", "Anom/Norm"], index=THRS_ARR)
    for thrs in THRS_ARR:
        Metrics, Med_Metrics = [], []
        Anom_Norm = []
        for y_true, y_prob in zip(CV_True_y, CV_Probs):
            y_pred = predict(y_prob, thrs)
#             Metrics.append(return_metrics(y_true, y_pred, y_prob))
            Med_Metrics.append(return_med_metrics(y_true, y_pred, y_prob))
            Anom_Norm.append([y_true.sum(), y_true.shape[0] - y_true.sum()])
#         DF.ix[thrs,:2] = np.mean(Metrics, axis=0)[:2]
        DF.loc[thrs,["Sensitivity", "Specificity","Medic F-Score", "ROC-AUC"]] = np.mean(Med_Metrics, axis=0)
        DF.loc[thrs,"Anom/Norm"] = "{}/{}".format(*map(int, np.mean(Anom_Norm, axis=0)))
    
    Best = DF.sort_values(by=['Medic F-Score'], ascending=False).iloc[0,:]
    
    log_to_print(Anom, Best)
#     log_to_file(Anom, Best, filename='_'.join([add_to_filename, 'XGBoost_Results.txt']))

For anomaly Density_a:

Чувствительность: 0.723
Специфичность: 0.763

Всего больных (в среднем на тесте): 36.1
Всего здоровых (в среднем на тесте): 48.9

Выявлено истинно больных : 26.1
Выявлено истинно здоровых : 37.3


For anomaly pH_a:

Чувствительность: 0.067
Специфичность: 0.936

Всего больных (в среднем на тесте): 1.9
Всего здоровых (в среднем на тесте): 83.0

Выявлено истинно больных : 0.1
Выявлено истинно здоровых : 77.7


For anomaly Protein_a:

Чувствительность: 0.733
Специфичность: 0.692

Всего больных (в среднем на тесте): 4.4
Всего здоровых (в среднем на тесте): 80.6

Выявлено истинно больных : 3.2
Выявлено истинно здоровых : 55.8


For anomaly Glucose_a:

Чувствительность: 0.425
Специфичность: 0.478

Всего больных (в среднем на тесте): 2.4
Всего здоровых (в среднем на тесте): 82.5

Выявлено истинно больных : 1.0
Выявлено истинно здоровых : 39.5


For anomaly Ketones_a:

Чувствительность: 0.345
Специфичность: 0.844

Всего больных (в среднем на тесте): 2.9
Всего здоровых (в

In [43]:
from statistics import harmonic_mean

In [49]:
harmonic_mean([0.007, 0.844])

0.013884841363102233

# Графическое представления подбора порога

In [38]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [39]:
@interact
def plot(Anom = ['Density_a', 'pH_a', 'Protein_a', 
                 'Glucose_a', 'Ketones_a', 'Leukocyte_a', 
                 'Nitrite_a', 'Urobilinogen_a', 'Blood_a', 
                 'Erythrocyte_a', 'Squamous cells_a', 
                 'Hyaline cylinders_a', 'Bacteria_a', 
                 'Crystals_a', 'Ferment_a', 'Small cells_a', 
                 'Pathological cylinders_a', 'Slime_a']):

    X = pd.concat([spectra.filter(regex='Ch_'), Gender_emb, Age_emb], axis=1).values
    y = spectra[Anom].astype(int).values
    groups = spectra.Dataset.values

    CV_True_y, CV_Probs = [], []
    gkf = GroupKFold()

    for train_inds, test_inds in gkf.split(X,y,groups):
        Train_X, Test_X = X[train_inds], X[test_inds]
        Train_y, Test_y = y[train_inds], y[test_inds]

        CLF = XGBClassifier(**Anom_Best_Dict[Anom][1]).fit(Train_X, Train_y)

        Probs = CLF.predict_proba(Test_X)[:,1]
        CV_Probs.append(Probs)
        CV_True_y.append(Test_y)
        
    THRS_ARR = np.arange(0.01,1.0,0.01)

    DF = pd.DataFrame(columns=["Precision", "Recall", 
                               "Sensitivity", "Specificity",
                               "Medic F-Score", "ROC-AUC", 
                                "Anom/Norm"], index=THRS_ARR)
    for thrs in THRS_ARR:
        Metrics, Med_Metrics = [], []
        Anom_Norm = []
        for y_true, y_prob in zip(CV_True_y, CV_Probs):
            y_pred = predict(y_prob, thrs)
            Metrics.append(return_metrics(y_true, y_pred, y_prob))
            Med_Metrics.append(return_med_metrics(y_true, y_pred, y_prob))
            Anom_Norm.append([y_true.sum(), y_true.shape[0] - y_true.sum()])
        DF.loc[thrs,["Precision", "Recall"]] = np.mean(Metrics, axis=0)[:2]
        DF.loc[thrs,["Sensitivity", "Specificity","Medic F-Score", "ROC-AUC"]] = np.mean(Med_Metrics, axis=0)
        DF.loc[thrs,["Anom/Norm"]] = "{}/{}".format(*map(int, np.mean(Anom_Norm, axis=0)))
    
    plt.figure(figsize=(10,6))
    plt.title(Anom, fontsize=14)
    plt.xlabel("Threshold of Prediction")
    plt.ylabel("Metric value")
    plt.plot(DF.index.values.round(2), DF.Sensitivity.values, label="Sensitivity", c='blue')
    plt.plot(DF.index.values.round(2), DF.Specificity.values, label="Specificity", c='orange')
    plt.plot(DF.index.values.round(2), DF['Medic F-Score'].values, label="Medic F-Score", c='green')
    
    plt.grid()
    plt.legend()
    plt.show()

interactive(children=(Dropdown(description='Anom', options=('Density_a', 'pH_a', 'Protein_a', 'Glucose_a', 'Ke…

# Посмотрим на важность признаков для разных аномалий

In [40]:
@interact
def plot(Anom = ['Density_a', 'pH_a', 'Protein_a', 
                 'Glucose_a', 'Ketones_a', 'Leukocyte_a', 
                 'Nitrite_a', 'Urobilinogen_a', 'Blood_a', 
                 'Erythrocyte_a', 'Squamous cells_a', 
                 'Hyaline cylinders_a', 'Bacteria_a', 
                 'Crystals_a', 'Ferment_a', 'Small cells_a', 
                 'Pathological cylinders_a', 'Slime_a']):

    X = pd.concat([spectra.filter(regex='Ch_'), Gender_emb, Age_emb], axis=1).values
    y = spectra[Anom].astype(int).values
    groups = spectra.Dataset.values

    CLF = XGBClassifier(**Anom_Best_Dict[Anom][1]).fit(Train_X, Train_y)
    
    Feature_Names = pd.concat([spectra.filter(regex='Ch_'), Gender_emb, Age_emb], axis=1).columns
    Feature_Importances = CLF.feature_importances_
    
    plt.figure(figsize=(12,7))
    plt.xticks(rotation=90)
    plt.ylabel("Feature importance", fontsize=12)
    plt.title(Anom, fontsize=14)
    plt.bar(Feature_Names, Feature_Importances)
    

interactive(children=(Dropdown(description='Anom', options=('Density_a', 'pH_a', 'Protein_a', 'Glucose_a', 'Ke…